In [8]:
import pennylane as qml
import numpy as np
import jax

jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
jnp = jax.numpy

dev = qml.device("default.qubit", wires=1)
H = 0.6 * qml.PauliZ(0) - 0.8 * qml.PauliY(0)


def qfunc(theta):
    qml.SpecialUnitary(theta, wires=0)
    return qml.expval(H)


circuit = qml.QNode(qfunc, dev, interface="jax", diff_method="parameter-shift")

theta = jnp.array([0.4, 0.2, -0.5])
circuit(theta)

Array(0.06674504, dtype=float64)

In [4]:
unit_vector = np.array([0.0, 1.0, 0.0])


def central_diff_grad(theta, delta):
    plus_eval = circuit(theta + delta / 2 * unit_vector)
    minus_eval = circuit(theta - delta / 2 * unit_vector)
    return (plus_eval - minus_eval) / delta


delta = 0.75
print(f"Central difference: {central_diff_grad(theta, delta):.5f}")

Central difference: 0.42398


In [43]:
import pennylane as qml
from jax import numpy as numpy
import jax

dev1 = qml.device("lightning.qubit",wires = 1)

@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0],wires=0)
    qml.RX(params[1],wires=0)
    return qml.expval(qml.PauliZ(0)) #qml.probs([0])#


0.5403023058597252

In [71]:
#params = [0.001,0.707]
params = np.array([0.011,0.012])
circuit(params)

dcircuit = jax.grad(circuit)
dcircuit(params)

def cost(x):
    return circuit(x)

cost(params)



In [106]:
dev = qml.device('default.qubit', wires = 2)

@qml.qnode(dev)
def trotterize(alpha, beta, time, depth):

    for i in range(depth):
        # qml.IsingXX(alpha*time/depth,[0,1])
        # qml.IsingZZ(beta*time/depth,[0,1])
        qml.IsingXX(2*alpha*time/depth,[0,1]) #+ qml.IsingZZ(beta*time/depth,[0,1])
        qml.IsingZZ(2*beta*time/depth,[0,1])

    
    # Return the probabilities
    return qml.probs()
trotterize(0.5,0.8,0.2,1)
#trotterize(0.9,1.0,0.4,2)

tensor([0.99003329, 0.        , 0.        , 0.00996671], requires_grad=True)

In [135]:
import json
import pennylane as qml
import pennylane.numpy as np

WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3

initial_params = np.random.random(NUM_PARAMETERS)

def variational_circuit(params,hamiltonian):
    """
    This is a template variational quantum circuit containing a fixed layout of gates with variable
    parameters. To be used as a QNode, it must either be wrapped with the @qml.qnode decorator or
    converted using the qml.QNode function.

    The output of this circuit is the expectation value of a Hamiltonian, somehow encoded in
    the hamiltonian argument

    Args:
        - params (np.ndarray): An array of optimizable parameters of shape (30,)
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value is returned.
    
    Returns:
        (float): The expectation value of the Hamiltonian
    """
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires = [0,1]))
circuit = qml.QNode(variational_circuit,dev)

def cost(x):
    return circuit(x)
cost(initial_params)


TypeError: variational_circuit() missing 1 required positional argument: 'hamiltonian'

In [132]:
def optimize_circuit(params,hamiltonian):
    """Minimize the variational circuit and return its minimum value.
    You should create a device and convert the variational_circuit function 
    into an executable QNode. 
    Next, you should minimize the variational circuit using gradient-based 
    optimization to update the input params. 
    Return the optimized value of the QNode as a single floating-point number.

    Args:
        - params (np.ndarray): Input parameters to be optimized, of dimension 30
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value you should minimize.
    Returns:
        float: the value of the optimized QNode
    """

    dev = qml.device("lightning.qubit",wires = 2)# Initialize the device.

    circuit = qml.QNode(variational_circuit,dev)# Instantiate the QNode from variational_circuit.
    
    # opt = qml.QNGOptimizer(0.01)
    # params_new = opt.step(circuit,initial_params)
    # #lol = qml.gradients.param_shift(circuit)(initial_params)
    return circuit#params_new
optimize_circuit(initial_params)

TypeError: optimize_circuit() missing 1 required positional argument: 'hamiltonian'

In [175]:
#Returning Expectation Values

def hamiltonian(num_wires):
    """
    A function for creating the Hamiltonian in question for a general
    number of qubits.

    Args:
        num_wires (int): The number of qubits.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    obs = []
    for i in range(num_wires):
        j = i+1
        while j < num_wires:
            obs.append(1/3*qml.X(i)@qml.X(j))
            j+=1
    for i in range(num_wires):
        obs.append(-qml.Z(i))
    coeffs=np.ones(len(obs))
    return qml.Hamiltonian(coeffs, obs)

hamiltonian(3)
#qml.Hadamard(num_wires)


(
    1.0 * ((0.3333333333333333 * X(0)) @ X(1))
  + 1.0 * ((0.3333333333333333 * X(0)) @ X(2))
  + 1.0 * ((0.3333333333333333 * X(1)) @ X(2))
  + 1.0 * -1 * Z(0)
  + 1.0 * -1 * Z(1)
  + 1.0 * -1 * Z(2)
)

In [296]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def circuit(weights):
    new_weights = weights.reshape(-1)
    #new_weights = new_weights.numpy()

    i = 0
    while i <= len(new_weights)-1:
        qml.RX(new_weights[i], wires = 0)
        qml.RY(new_weights[i+1], wires = 1)
        qml.RZ(new_weights[i+2], wires = 2)
        i += 3
        qml.broadcast(qml.CNOT, wires=[0, 1, 2], pattern="ring")
        # qml.CNOT(wires=[0,1])
        # qml.CNOT(wires=[1,2])
        # qml.CNOT(wires=[2,0])
    
    obs = qml.Y(0) @ qml.I(1) @ qml.Z(2)
    return qml.expval(obs)

weights = np.array([[1,0.5,-0.765],[0.1,0,-0.654]])
# key = jax.random.PRNGKey(42)
# params = jax.random.normal(key,[6])
# circuit(params)
circuit(weights)
#weights.numpy()
# qml.draw_mpl(circuit, decimals=2)(params)

elem = weights.reshape(-1)
elem[0] += 1
elem

tensor([ 2.   ,  0.5  , -0.765,  0.1  ,  0.   , -0.654], requires_grad=True)

In [313]:
def parameter_shift(weights):   
    new_weights = weights.reshape(-1)
    s = 0.5
    def gradient(weights,idx):
        weights_1 = weights.copy()
        weights_1[idx] += s
        plus_eval = circuit(weights_1)
        weights_1[idx] -= 2*s
        minus_eval = circuit(weights_1)
        return (plus_eval-minus_eval)/(2*np.sin(s))

    grad_lst = []

    for i in range(len(new_weights)):
        grad = gradient(new_weights,i)
        grad_lst.append(grad)
    grad_lst = np.array(grad_lst)

    return grad_lst.numpy().reshape(2,3)
    # #return np.array(u[2])
    #return grad_lst
weights_1= np.array([[0.94,-0.2,6.03],[-2.6,-0.058,1.2]])
# arr = np.array(parameter_shift(weights_1))
# arr.numpy().reshape(2,3)

parameter_shift(weights_1)

array([[ 2.99518791e-02, -3.93131888e-02, -5.78934024e-17],
       [-3.40899969e-02,  1.66279776e-01, -2.31573610e-16]])

In [279]:
s = 0.5
key = jax.random.PRNGKey(42)
params = jax.random.normal(key,[6])
def parameter_shift_term(qnode,params,i):
    params.reshape(-1)
    #params = params.numpy()
    shifted = params.copy()
    shifted = shifted.at[i].add(s)
    forward = qnode(shifted)
    return params#shifted
    #forward = qnodes
parameter_shift_term(circuit,params,0)
type(params)


jaxlib.xla_extension.ArrayImpl